In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import split_dataset_by_date, clean_historical_data, check_tickers_for_missing_values
from utilities import calc_vif, calc_p_values, calc_correlation, highlight_vif, highlight_p_values, evaluate_regression_model, evaluate_cross_validation
from utilities import load_data, save_data

In [2]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Machine learning models (regression)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Ensemble methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
file_name = "sp500_adj_close_raw_with_nas"
file_path = f"../../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║File `test_w_na.csv.bz2` loaded from `sp500_adj_close_raw_with_nas.zip`║
╚═══════════════════════════════════════════════════════════════╝


### Data Pre-Processing
___

#### Split todays data (For prediction) and historical data (For training)

In [4]:
todays_date = "2024-10-25"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-25


#### Ensure there are missing values on Todays Data is what we are predicting...

In [5]:
todays_data.isnull().sum()

Date                    0
Ticker                  0
Adjusted Close          0
Next Day Close        501
Previous Day Close      0
Return                  0
Volatility              0
RSI                     0
SMA_50                  0
SMA_100                 0
SMA_200                 0
Upper Band              0
Lower Band              0
Support                 0
Resistance              0
Action                501
dtype: int64

#### Handle missing values (NA's) on the historical data used to train and test...

In [6]:
historical_data = clean_historical_data(historical_data)

historical_data.isnull().sum()

Date                  0
Ticker                0
Adjusted Close        0
Next Day Close        0
Previous Day Close    0
Return                0
Volatility            0
RSI                   0
SMA_50                0
SMA_100               0
SMA_200               0
Upper Band            0
Lower Band            0
Support               0
Resistance            0
Action                0
dtype: int64

In [7]:
tickers_no_missing_values, tickers_with_missing_values = check_tickers_for_missing_values(historical_data)

╔═══════════════════════════════════════════════════════════════╗
║          Tickers that do not have any missing values          ║
╠═══════════════════════════════════════════════════════════════╣
║ Number of unique tickers:      |          501.00000           ║
║ Number of tickers with no m... |          501.00000           ║
║ Number of tickers with miss... |           0.00000            ║
╚═══════════════════════════════════════════════════════════════╝


### Exploratory Data Analysis (EDA):
___

#### Todays Data:

In [8]:
print("Today's Data Shape:", todays_data.shape)

Today's Data Shape: (501, 16)


#### Historical Data:

In [9]:
print("Historical Data Shape:", historical_data.shape)

Historical Data Shape: (1978479, 16)


### Create Multiple Versions of Dataset
___

#### Select which version of the data to work with

In [10]:
# Data with dates and without tickers (Set as index for reference)
def prepare_data_v2(main_data: pd.DataFrame) -> pd.DataFrame:
    # Create a copy of the input DataFrame
    df = main_data.copy().reset_index(drop=True)
    
    # Convert the `Date` column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from the `Date` column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Set the index to `Date` and `Ticker`
    df = df.set_index(["Date", "Ticker"])
    
    return df

main_data = prepare_data_v2(historical_data.copy().reset_index(drop=True))

##### **Note: It is recommended to remove `["Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]` after VIF inspection...**

##### **Note: It is recommended to remove `["Day"]` after p-value inspection...**

In [11]:
select_data = main_data.copy()

select_columns_to_drop = ["Action", "Previous Day Close", "SMA_50", "Resistance", "Upper Band", "SMA_200", "Day"]

data = select_data.drop(columns=select_columns_to_drop)

print("Shape:", data.shape)
data.head()

Shape: (1978479, 10)


Adjusted Close  Next Day Close    Return  Volatility  \
Date       Ticker                                                         
2008-01-02 A            23.256388       23.025743 -0.011976    0.015704   
           AAPL          5.876342        5.879055 -0.016357    0.018937   
           ABT          18.130209       18.019754 -0.006055    0.010484   
           ACGL          7.608889        7.764444 -0.026581    0.016022   
           ACN          26.437077       25.982521  0.000833    0.024039   

                         RSI    SMA_100  Lower Band    Support  Year  Month  
Date       Ticker                                                            
2008-01-02 A       48.827590  23.299887   22.540236  21.392029  2008      1  
           AAPL    59.067350   4.939064    5.403559   4.637376  2008      1  
           ABT     34.677372  17.628250   18.221804  16.775562  2008      1  
           ACGL    45.154190   7.878933    7.378535   7.463333  2008      1  
           ACN     54.812176  27.784420   24.273775  24.765512  2008      1

### Split data features `X` and target `y`
___

In [12]:
# Split the data into features (X) and target (y)
X = data.drop(columns="Next Day Close")

y = data["Next Day Close"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (1978479, 9)
Shape of y: (1978479,)


### Data Splitting
___

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, # 80% training and 20% testing
    random_state=42
)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1582783, 9)
Shape of X_test: (395696, 9)
Shape of y_train: (1582783,)
Shape of y_test: (395696,)


### Scaling the Data
___

#### Scale the data using `StandardScaler`

In [14]:
# Scale using StandardScaler
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

X_train_scaled shape: (1582783, 9)
X_test_scaled shape: (395696, 9)
y_train_scaled shape: (1582783, 1)
y_test_scaled shape: (395696, 1)


### Model Training
___


In [15]:
from sklearn.model_selection import GridSearchCV

# XGBoost Regressor
xgb_model = XGBRegressor(
    tree_method="hist", # Use GPU for training
    device="cuda", # Use GPU for training
    random_state=42 
)

xgb_param_grid = {
    "colsample_bytree": [0.5, 0.7], # Subsample ratio of columns when constructing each tree
    "gamma": [0, 0.1], # Minimum loss reduction required to make a further partition on a leaf node of the tree
    "learning_rate": [0.01, 0.1], # Step size shrinkage used to prevent overfitting
    "max_depth": [3, 5], # Maximum depth of the tree
    "n_estimators": [100, 200], # Number of trees
    "subsample": [0.5, 0.7], # Subsample ratio of the training instances
}

grid_search_xgb = GridSearchCV(
    estimator=xgb_model, # XGBoost Regressor
    param_grid=xgb_param_grid, # Parameter grid
    cv=3, # Number of folds
    scoring="neg_mean_absolute_error", # Scoring metric
    # n_jobs=-1, # Use all available processors
    verbose=2 # Print detailed information
)

grid_search_xgb.fit(X_train_scaled, y_train_scaled)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


c:\Users\chris\anaconda3\envs\dev\lib\site-packages\xgboost\core.py:158: UserWarning: [19:58:14] WARNING: D:\bld\xgboost-split_1727635012356\work\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.5; total time=   1.0s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.5; total time=   0.7s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.5; total time=   0.7s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.7s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.7s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.7s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.5; total time=   1.2s
[CV] END colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.5; total time=   1.2s


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device='cuda',
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             param_grid={'colsample_bytree': [0.5, 0.7], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 5],
                         'n_estimators': [100, 200], 'subsample': [0.5, 0.7]},
             scoring='neg_mean_absolute_error', verbose=2)

In [16]:
import json

print(json.dumps(grid_search_xgb.best_params_, indent=4))

{
    "colsample_bytree": 0.7,
    "gamma": 0,
    "learning_rate": 0.1,
    "max_depth": 5,
    "n_estimators": 200,
    "subsample": 0.7
}


### Model Evaluation
___

In [17]:
best_xgb_model = grid_search_xgb.best_estimator_

In [20]:
# Predict the target values using the testing data
y_predict_scaled = best_xgb_model.predict(X_test_scaled)
y_train_predict_scaled = best_xgb_model.predict(X_train_scaled)

# Unscale the testing data and predictions
y_predict_unscaled = y_scaler.inverse_transform(y_predict_scaled.reshape(-1, 1))
y_test_unscaled = y_scaler.inverse_transform(y_test_scaled)

# Unscale the training data and predictions
y_train_predict_unscaled = y_scaler.inverse_transform(y_train_predict_scaled.reshape(-1, 1))
y_train_unscaled = y_scaler.inverse_transform(y_train_scaled)

In [21]:
evaluate_regression_model(
    best_xgb_model,
    "XGBoost Regressor",
    X_train_scaled,
    y_train_scaled,
    X_test_scaled,
    y_test_scaled,
    y_train_unscaled,
    y_test_unscaled,
    y_train_predict_unscaled,
    y_predict_unscaled
)

╔═══════════════════════════════════════════════════════════════╗
║              XGBoost Regressor Model Evaluation               ║
╠═══════════════════════════════════════════════════════════════╣
║                                |                              ║
║ Training Data Metrics          |                              ║
║ Mean Squared Error (Train):    |         4,161.41641          ║
║ Root Mean Squared Error (Tr... |           64.50904           ║
║ R-Squared (Train):             |           0.89611            ║
║ Adjusted R-Squared (Train):    |           0.89611            ║
║ Mean Absolute Error (Train):   |           4.36939            ║
║ Mean Absolute Percentage Er... |           2.69467            ║
║                                |                              ║
║ Testing Data Metrics           |                              ║
║ Mean Squared Error (Test):     |         6,989.67448          ║
║ Root Mean Squared Error (Te... |           83.60427           ║
║ R-Square

In [22]:
cv_scores = cross_val_score(
    best_xgb_model,
    X_train_scaled,
    y_train_scaled,
    cv=5,
    scoring="r2"
)

In [23]:
evaluate_cross_validation(cv_scores, "XGBoost Regressor")

╔═══════════════════════════════════════════════════════════════╗
║           XGBoost Regressor Cross Validation Scores           ║
╠═══════════════════════════════════════════════════════════════╣
║ Fold 1:                        |           0.85087            ║
║ Fold 2:                        |           0.86078            ║
║ Fold 3:                        |           0.86427            ║
║ Fold 4:                        |           0.84942            ║
║ Fold 5:                        |           0.85785            ║
║                                |                              ║
║ Mean R^2 Score:                |           0.85664            ║
║ Standard Deviation:            |           0.00570            ║
╚═══════════════════════════════════════════════════════════════╝


In [25]:
import pickle

# Save model path
model_name = "XGBRegressor.pkl"
model_path = f"../../../models/{model_name}"

# Save the model to a file
with open(model_path, "wb") as file:
    pickle.dump(best_xgb_model, file)

print(f"Model saved to {model_path}")

Model saved to ../../../models/XGBRegressor.pkl


### Predict tomorrows `Adjusted Close`
___
___
___

In [26]:
def preprocess_todays_data(df, columns_to_drop):
    # Convert 'Date' column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from 'Date' column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Drop specified columns
    df = df.drop(columns=columns_to_drop)
    
    # Set 'Date' and 'Ticker' as the index
    df = df.set_index(["Date", "Ticker"])
    
    # Drop 'Next Day Close' column
    df = df.drop(columns="Next Day Close")
    
    return df

X_to_predict = todays_data.copy()

X_to_predict = preprocess_todays_data(X_to_predict, select_columns_to_drop)

X_to_predict_scaled = X_scaler.transform(X_to_predict)

print("Shape of todays_data_clean_scaled:", X_to_predict_scaled.shape)

Shape of todays_data_clean_scaled: (501, 9)


In [27]:
y_to_predict_scaled = best_xgb_model.predict(X_to_predict_scaled)

y_to_predict_unscaled = y_scaler.inverse_transform(y_to_predict_scaled.reshape(-1, 1))

print("Shape of y_to_predict_unscaled:", y_to_predict_unscaled.shape)

Shape of y_to_predict_unscaled: (501, 1)


In [28]:
prediction_df = todays_data.copy()

prediction_df["Next Day Close"] = y_to_predict_unscaled.round(3)

prediction_df = prediction_df[["Date", "Ticker", "Adjusted Close", "Next Day Close"]]

prediction_df = prediction_df.set_index(["Date", "Ticker"])

print("Shape:", prediction_df.shape)
display(prediction_df.head(20))
display(prediction_df.tail(20))

Shape: (501, 2)


Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 A              130.160      130.714996
           AAPL           231.400      234.897003
           ABBV           187.820      186.307999
           ABNB           134.600      134.709000
           ABT            114.240      114.432999
           ACGL           105.280      105.343002
           ACN            360.790      358.028015
           ADBE           483.730      510.875000
           ADI            230.170      225.906998
           ADM             56.570       56.719002
           ADP            287.960      289.881989
           ADSK           286.050      288.471008
           AEE             87.800       87.679001
           AEP             98.180       98.091003
           AES             16.965       16.830000
           AFL            109.710      109.904999
           AIG             76.820       78.000000
           AIZ            191.780      193.220001
           AJG            283.280      287.800995
           AKAM           101.980      102.127998

Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 WBD              7.590        7.527000
           WDC             69.455       68.667000
           WEC             97.170       96.750999
           WELL           130.190      129.876999
           WFC             64.520       64.546997
           WM             206.800      208.123001
           WMB             52.510       53.423000
           WMT             82.510       83.210999
           WRB             58.170       58.603001
           WST            307.920      307.705994
           WTW            290.410      289.501007
           WY              31.785       32.556000
           WYNN            97.980       96.002998
           XEL             64.460       64.129997
           XOM            119.510      120.385002
           XYL            130.410      130.182999
           YUM            133.040      132.483994
           ZBH            102.330      102.853996
           ZBRA           359.970      356.968994
           ZTS            179.910      184.128006

In [29]:
# Make index column a regular column
prediction_df.reset_index(inplace=True)

prediction_df

,Date,Ticker,Adjusted Close,Next Day Close
0,2024-10-25,A,130.16,130.714996
1,2024-10-25,AAPL,231.40,234.897003
2,2024-10-25,ABBV,187.82,186.307999
3,2024-10-25,ABNB,134.60,134.709000
4,2024-10-25,ABT,114.24,114.432999
...,...,...,...,...
496,2024-10-25,XYL,130.41,130.182999
497,2024-10-25,YUM,133.04,132.483994
498,2024-10-25,ZBH,102.33,102.853996
499,2024-10-25,ZBRA,359.97,356.968994


In [30]:
file_name = "XGB_predict.zip"
file_path = f"../../../data/raw_data/{file_name}"

save_data(prediction_df, file_path)

╔═══════════════════════════════════════════════════════════════╗
║   File `XGB_predict.zip` already exists. Overwriting file.    ║
╚═══════════════════════════════════════════════════════════════╝
╔═══════════════════════════════════════════════════════════════╗
║          File saved and zipped as `XGB_predict.zip`           ║
╚═══════════════════════════════════════════════════════════════╝
